In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available: False
Device: CPU


In [3]:
from fastai.vision.all import *
import random, torch

path = untar_data(URLs.PETS)

def is_cat(x):
    return x[0].isupper()

# Take a small subset so it runs fast even on CPU
files = get_image_files(path/"images")
random.seed(42)
files = random.sample(files, 800)   # tweak: 300–1200 depending on speed

dls = ImageDataLoaders.from_name_func(
    path,
    files,
    valid_pct=0.2,
    seed=42,
    label_func=is_cat,
    item_tfms=Resize(160)  # smaller size = faster
)

learn = vision_learner(dls, resnet18, metrics=accuracy)

# Remove progress callback (your earlier issue)
for cb in list(learn.cbs):
    if "Progress" in cb.__class__.__name__:
        learn.remove_cb(cb.__class__)

# Train just 1 epoch (fast)
learn.fine_tune(1, base_lr=3e-3)

img = files[0]
pred, pred_idx, probs = learn.predict(img)
print("Sample image:", img.name)
print("Prediction:", pred, "| Confidence:", float(probs[pred_idx]))


[0, 0.7300056219100952, 0.1435271054506302, 0.9437500238418579, '01:22']
[0, 0.17774705588817596, 0.2040298730134964, 0.925000011920929, '01:43']
Sample image: Bombay_173.jpg
Prediction: True | Confidence: 0.999994158744812
